# The CMNN Photoz Estimator
A Python3 implementation of the Color-Matched Nearest-Neighbors (CMNN) photometric redshift estimator.<br>
Melissa L. Graham and Andrew J. Connolly, 2020. <br>

**Last Updated:** Feb 2023

**Package Dependencies:** os, datetime, argparse, matplotlib, numpy, and scipy.

**Last verified to run** with python 3.10.9, numpy 1.21.6, scipy 1.9.3, and matplotlib 3.6.2. 

**This estimator has been applied in:**
 * ["Photometric Redshifts with the LSST: Evaluating Survey Observing Strategies", Graham et al. (2018)](https://ui.adsabs.harvard.edu/abs/2018AJ....155....1G/abstract)
 * ["Photometric Redshifts with the LSST. II. The Impact of Near-infrared and Near-ultraviolet Photometry", Graham et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020AJ....159..258G/abstract)

## 1. Overview

The color-matched nearest-neighbors (CMNN) photometric redshift (photo-_z_) estimator provides photometric redshifts with an accuracy that is directly dependent on (and only on) the quality of the simulated observed photometry which, as described below, depends directly and only on the $5{\sigma}$ limiting magnitudes (i.e., the LSST survey depths).
This makes it an ideal tool for assessing the impact on photo-_z_ results from different proposed LSST observing strategies that build image depth over time in different ways, or result in different coadded depths.

### 1.1. Mock catalog for test and training sets

The provided mock catalog (file `mock_catalog.dat`) is described in Graham et al. (2020).
It contains simulated galaxies redshifts and magnitudes, and is limited to galaxies with redshifts $<$3.5 and _i_-band magnitudes between 16.0 and 25.3 mag.
This matches the $i\leq25$ mag "gold sample" defined by the Science Requirement Document (ls.st/lpm-17), 
and matches many of the photo-_z_ simulations in the papers listed above.
**Please get in touch if you need a catalog that is deeper in the _i_-band.**

The catalog columns include a unqiue index, the "true" redshift, and the "true" apparent magnitude in LSST (SDSS) _ugrizy_ magnitudes, plus a Euclid _Y_ band, and _JHK_.
Although 10 bands are provided, only nine are used for the test and training sets, as the user must choose between LSST y (default) or Euclid _Y_.

The user specifies the size of the test and training sets, and the desired 5-sigma magnitude depth in each filter (i.e., for a given observing strategy).
The observed apparent magnitudes and uncertainties for all filters are then simulated, based on the "true" magnitudes and the desired depths, as described in the papers above.

The photometric error, $\sigma_{\rm rand}$, is given in Section 3.2.1 of [Ivezić et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...873..111I/abstract):
$\sigma^2_{\rm rand} = (0.04-\gamma)x + \gamma x^2$, where $x=10^{0.4(m-m_5)}$, 
$m_5$ is the $5\sigma$ limiting magnitude, 
$m$ is the magnitude of the galaxy, 
and for the LSST optical filters the values for $\gamma$, which sets the impact of, e.g., sky brightness, are 
$0.037$, $0.038$, $0.039$, $0.039$, $0.04$, and $0.04$ for filters _ugrizy_, respectively.
Values of $0.04$ are also used for _JHK_ bands.

A random value drawn from a normal distribution with a standard deviation equal to the photometric error for each galaxy is added to the true catalog magnitude to generate observed apparent magnitudes.
For example, the photometric error of a galaxy with a true catalog magnitude of $i=24.5$ mag would be $\sigma_{\rm rand} = 0.026$; a random normal draw of $0.054$ would lead to an observed apparent magnitude of $i=24.55$ mag.
Observed apparent colors are calculated from the observed apparent magnitudes, and the error in the color is the photometric errors added in quadrature.

In this way, test and training sets are simulated based on a given observing strategy and the depth it would achieve in each filter.

### 1.2. Estimating photometric redshifts

For each galaxy in the test set, the estimator identifies a color-matched subset of training galaxies by calculating the 
Mahalanobis distance in color-space between the test galaxy and all training galaxies:

$D_M = \sum_{\rm 1}^{N_{\rm colors}} \frac{( c_{\rm train} - c_{\rm test} )^2}{ (\delta c_{\rm test})^2}$

where $c$ is the color of the test- or training-set galaxy, 
$\delta c_{\rm test}$ is the uncertainty in the test galaxy's color, 
and $N_{\rm color}$ is the number of colors measured for both the test- and training-set galaxy. 

A threshold value is then applied to all training-set galaxies to identify those which are well-matched in color, 
as defined by the percent point function (PPF): e.g., if the number of degrees of freedom $N_{\rm color}=5$, 
PPF$=68\%$ of all training galaxies consistent with the test galaxy will have $D_M < 5.86$.

A training galaxy is then selected by one of three modes from this subset of color-matched nearest-neighbors, 
and its redshift is used as the test-set galaxy's photometric redshift.

The three modes of selection are to choose randomly, to choose the galaxy with the lowest $D_M$, or to weight the random selection by the inverse of the $D_M$.
The standard deviation in redshifts of this subset of training galaxies is used as the uncertainty in the photo-_z_ estimate.

### 1.3. Analysis with statistical measures

Three types of plots are automatically created for each run:
 * (1) true redshift vs. photometric redshift (tzpz)
 * (2) statistical measures based on the photo-_z_ accuracy, $\Delta z = (|z_{\rm true} - z_{\rm phot}|)/(1+z_{\rm phot})$, in bins of photometric redshift:
   * (a) standard deviation
   * (b) bias
   * (c) fraction of outliers
 * (3) histograms representing the training set

Plots comparing the statistical measures for different runs can be created by users with the `cmnn_analysis` module's `make_stats_plots` function.

## 2. Examples

### 2.1. Photo-z results based on LSST photometry by year

#### 2.1.1. Obtain the 5$\sigma$ limiting magnitudes for years 1, 3, 6, and 10

Use the `cmnn_tools.py` module to get the limiting magnitues for a baseline survey.
**This is the OLD baseline observing strategy!!**

```
path/CMNN_Photoz_Estimator: python
>>> import cmnn_tools
>>> for yr in [1,3,6,10]:
...     print( cmnn_tools.convert_year_to_depths_baseline(yr))
...
[ 24.83523503  26.12886248  26.28102228  25.58102228  24.80514998  23.60514998]
[ 25.4316366   26.72526405  26.87742385  26.17742385  25.40155155  24.20155155]
[ 25.8079241   27.10155155  27.25371134  26.55371134  25.77783904  24.57783904]
[ 26.08523503  27.37886248  27.53102228  26.83102228  26.05514998  24.85514998]
```

#### 2.1.2. Run the CMNN Estimator for year 1

For the test set, use the year 1 magnitude limits above
as both the 5-sigma depths (`test_m5`, which sets photometric quality)
and the magnitude cuts (`test_mcut`, which defines what is "detected").

Assume that by year 1, a 10-year depth "training set" has been built during commissioning.
For the training set, use the year 10 magnitude limits above
as both the 5-sigma depths (`train_m5`, which sets photometric quality)
and the magnitude cuts (`train_mcut`, which defines what is "detected").

Since the NIR bands are not being used here, set their `filtmask` to 0 and use a placeholder of 24.0 mag.

For both the test and training sets, **limit to _i_-band magnitude cuts of 25 mag**.
Leave all other user-specified arguements as their default values.

```
python cmnn_run.py --verbose True --runid lsst-year1 \
--filtmask   1 1 1 1 1 1 0 0 0 \
--test_m5    24.84 26.13 26.28 25.58 24.81 23.61 24.0 24.0 24.0 \
--test_mcut  24.84 26.13 26.28 25.00 24.81 23.61 24.0 24.0 24.0 \
--train_m5   26.09 27.38 27.53 26.83 26.06 24.86 24.0 24.0 24.0 \
--train_mcut 26.09 27.38 27.53 25.00 26.06 24.86 24.0 24.0 24.0
```

Repeat for LSST years 3, 6, and 10.

It is also possible to set up a script to run all four at once -- see the file `script_example.sh`.

#### 2.1.3. View the results

The default statistical measure plots per run are in the individual output directories, e.g., `output/run_lsst-year1/analysis/*.png`.

Make plots comparing the statististical measures for all years and view them.

```
python
>>> import cmnn_analysis
>>> cmnn_analysis.make_stats_plots( \
multi_run_ids = ['lsst-year1','lsst-year3','lsst-year6','lsst-year10'], \
multi_run_labels = ['Year 1','Year 3','Year 6','Year 10'])
```

Now view the statistical comparison plots in `output/stats_plots/`:
 * CORIQRbias_year1_year3_year6_year10.png
 * CORIQRstdd_year1_year3_year6_year10.png
 * fout_year1_year3_year6_year10.png

For example, the combined plot for robust standard deviation should look similar to this.
Note that especially the high-_z_ bin values can fluctuate when a low number of test-set galaxies are used, and that the default for N\_test is only 40000 (Table 1).

<img src="example.png" alt="example" style="width: 400px;"/>

### 2.2. LSST at 10-years with and without Euclid or Roman photometry

TBD

## 3. Summaries of all modules and functions

### 3.1. `cmnn_run`

The only module that can parse user input from the terminal command line.

Runs the other three main modules: `cmnn_catalog`, `cmnn_photoz`, `cmnn_analysis`.

#### 3.1.1. `cmnn_run.main`
 * Parses the command line input. 
 * Checks that input values are good. 
 * Creates an output directory for the run.
 * Writes a file containing the input values to the output directory. 
 * Passes input to the function `cmnn_run.run`.

##### Table 1: User specified inputs

| Inputs | Type | Default | Description |
| :--    | :--  | :--     | :--         |
| verbose        | bool     | True                          | if True, prints optional info as standard output |
| runid          | str      | myrunid                       | unique run identifier for labeling the output files |
| catalog        | str      | mock_catalog.dat              | local filename of the mock galaxy catalg |
| clobber        | bool     | False                         | if True, overwrites any existing output for this runid |
| filtmask       | int[6]   | 1 1 1 1 1 1 1 1 1             | mask for use of filters u g r i z y J H K |
| yfilt          | int      | 0                             | y-band filter: 0=PanSTARRS or 1=Euclid |
| smart_nir      | int      | 0                             | if 1, only use NIR mags they reduce the pz uncertainty |
| roman_exp      | int      | 0                             | enables experiments with a single Roman filter |
| test\_m5       | float[6] | 26.1 27.4 27.5 26.8 26.1 24.9 | the 5-sigma magnitude limits (depths) to apply to the test-set galaxies |
| train\_m5      | float[6] | 26.1 27.4 27.5 26.8 26.1 24.9 | the 5-sigma magnitude limits (depths) to apply to the training-set galaxies |
| test\_mcut     | float[6] | 26.1 27.4 27.5 25.0 26.1 24.9 | a magnitude cut-off to apply to the test-set galaxies |
| train\_mcut    | float[6] | 26.1 27.4 27.5 25.0 26.1 24.9 | a magnitude cut-off to apply to the training-set galaxies |
| force\_idet    | bool     | True                          | if True, force detection in _i_-band for all test and train galaxies |
| force\_gridet  | bool     | True                          | if True, force detection in _g_, _r_, and _i_-bands for all test and train galaxies |
| test\_N        | int      | 40000                         | number of test-set galaxies | 
| train\_N       | int      | 200000                        | number of training-set galaxies | 
| cmnn\_minNc    | int      | 3                             | minimum number of colors required for inclusion in catalog |
| cmnn\_minNN    | int      | 10                            | forced minimum number of training-set galaxies in the CMNN subset |
| cmnn\_ppf      | float    | 0.68                          | the percent point function that defines the Mahalanobis distance threshold of the CMNN |
| cmnn\_rsel     | int      | 2                             | mode of random selection from the CMNN subset (0 = random; 1 = nearest neighbor; 2 = random weighted by inverse of Mahalanobis distance) |
| cmnn\_ppmag    | bool     | False                         | if True, apply a "pseudo-prior" based on the test-set's i-band magnitude (can only be True if force\_idet=True) |
| cmnn\_ppclr    | bool     | True                          | if True, apply a "pseudo-prior" based on the test-set's g-r and r-i color (can only be True if force\_gridet=True) |
| stats\_COR     | float    | 1.5                           | catastrophic outlier rejection; reject galaxies with $(z_{true}-z_{phot}) >$ `stats_COR` from the statistical measures of standard deviation and bias |

**Notes:**

The default limiting magnitudes are the 10-year 5$\sigma$ limiting magnitude depths for a baseline 
observing strategy which accumulates 56, 80, 184, 184, 160, 160 visits in filters _ugrizy_ 
(both `m5` and `mcut`, except for _i_-band where `mcut`=25).

The `roman_exp` argument is really just for Melissa, check the codes if you want to see what this is doing.

#### 3.1.2. `cmnn_run.run`

Called by `cmnn_run.main`.

Passes user-specified input to the following modules, in order:
 * `cmnn_catalog.make_test_and_train`
 * `cmnn_catalog.make_plots`
 * `cmnn_photoz.make_zphot`
 * `cmnn_analysis.make_stats_file`
 * `cmnn_analysis.make_stats_plots`
 * `cmnn_analysis.make_tzpz_plot`
 * `cmnn_analysis.make_hist_plots`

Appends the date and time of each stage to `output/run\__runid_/timestamps.dat`.

### 3.2. `cmnn_catalog`

#### 3.2.1. `cmnn_catalog.make_test_and_train`

Called by `cmnn_run.run`.

**What it does::** Simulates observed apparent magnitudes for test and training sets based on the user-specified 5$\sigma$ limiting magnitudes (test\_m5 and train\_m5), and applies the user-specified cuts (test\_mcut, train\_mcut). Chooses randomly from the full mock catalog to create user-specified number of test and training set galaxies.

**Exit Condition:** If the user input sizes for the test and training sets are larger than what the catalog can support, given the user-supplied magnitude depths and cuts, an error message will be returned and the code will exit without writing. Unfortunately the compatibility of the magnitude depths and cuts with the desired sizes of the test and training set cannot be tested in cmnn\_run.py.

**Outputs:** Writes data files of photometry to output/run\__runid_/test.cat and train.cat.

#### 3.2.2. `cmnn_catalog.make_plots`

Called by `cmnn_run.run`.

**What it does:** Generates histograms of the redshifts and apparent magnitudes for the test and training sets, and plots the apparent magnitude error vs. the apparent magnitude for all filters, for the test and training sets.

**Outputs:** Saves plots to the output/run\__runid_/plot\_cats/ directory (hist\_ztrue.png, hist\_mag.png, hist\_mage.png, test\_mag\_vs\_mage.png, and train\_mag\_vs\_mage.png).


### 3.3. `cmnn_photoz`

#### 3.3.1. `cmnn_photoz.make_zphot`

Called by `cmnn_run.run`.

**What it does:** Esimates photometric redshifts using the test and training sets of a given run.

**Outputs:** Writes test-set photo-z file to output/run\__runid_/zphot.cat.

#### 3.3.2. `cmnn_photoz.return_photoz`

Called by `cmnn_zphot.make_zphot`.

**What it does:** For a given test-set galaxy and a given training-set of galaxies, estimates the photo-_z_ of the test-set galaxy. 

**Returns:** A three-element array containing the estimated photo-_z_, the photo-_z_ error, and the number of color-matched nearest neighbours identified from the training set.


### 3.4. `cmnn_analysis`

#### 3.4.1. `cmnn_analysis.make_stats_file`

Called by `cmnn_run.run`.

**What it does:** Calculates the photo-z statistics in bins of photo-_z_.
The default is overlapping bins: [0.00,0.30],[0.15,0.45],[0.30,0.60], ... [2.70,3.00].
The photo-_z_ bins cannot be set by the user from `cmnn_run.run`, but when calling the module directly, input\_zbins can be passed.
The user can choose to use bins of true redshift by setting the input parameter bin\_in\_truez equal to True (default is False, to bin in photo-_z_).
Statistical measures are based on the photo-$z$ error: $\Delta z_{1+z_p} = (z_t-z_p)/(1+z_p)$ where $z_t$ is the true redshift, and $z_p$ the photo-$z$, of the test-set galaxy.
For some statistics, catastrophic outlier rejection (COR) is done first, and test galaxies in the bin with $|z_t-z_p| >$ stats\_COR (default 1.5) are rejected.
Outliers are defined as in the Science Requirements Document: $|\Delta z_{1+z_p}| > 3\sigma_{\rm IQR}$ _and_ $|\Delta z_{1+z_p}| > 0.06$. 

**Outputs:** Writes statistics to file output/run\__runid_/analysis/stats.dat. Columns of this file are described below. Will instead write to output/run\__runid_/analysis/stats_truezbins.dat if the user input parameter bin\_in\_truez is set to True (default is False, default to bin in photo-z).


##### Table 3: Statistical Measures 
| Col | Column Name | Description |
| :-- | :-- | :-- |
|  0 | zlow | photo-z bin lower limit |
|  1 | zhi  | photo-z bin upper limit |
|  2 | meanz    | the mean zphot of galaxies in bin |
|  3 | CORmeanz | post-COR mean zphot of galaxies in bin |
|  4 | fout    | fraction of outliers |
|  5 | CORfout | post-COR fraction of outliers |
|  6 | stdd    | standard deviation in $\Delta z_{1+z_p}$ of all galaxies in bin |
|  7 | bias    | mean $\Delta z_{1+z_p}$ of all galaxies in bin |
|  8 | outbias | mean $\Delta z_{1+z_p}$ of all outlier galaxies in bin |
|  9 | IQR     | interquartile range of $\Delta z_{1+z_p}$ |
| 10 | IQRstdd | stdandard deviation from the IQR ( = IQR / 1.349 ) |
| 11 | IQRbias | bias of test galaxies in the IQR  |
| 12 | CORstdd     | post-COR stdd |
| 13 | CORbias     | post-COR bias |
| 14 | CORoutbias  | post-COR outlier bias |
| 15 | CORIQR      | post-COR IQR |
| 16 | CORIQRstdd  | post-COR IQR stdd |
| 17 | CORIQRbias  | post-COR IQR bias |
| 18 | estdd       | error in stdd |
| 19 | ebias       | error in bias |
| 20 | eoutbias    | error in outlier bias |
| 21 | eIQR        | error in IQR |
| 22 | eIQRstdd    | error in IQR stdd |
| 23 | eIQRbias    | error in IQR bias |
| 24 | eCORstdd    | error in post-COR stdd |
| 25 | eCORbias    | error in post-COR bias |
| 26 | eCORoutbias | error in post-COR outlier bias |
| 27 | eCORIQR     | error in post-COR IQR |
| 28 | eCORIQRstdd | error in post-COR IQR stdd |
| 29 | eCORIQRbias | error in post-COR IQR bias |

#### 3.4.2. `cmnn_analysis.make_stats_plots`

Called by `cmnn_run.run`.

**What it does:** Plots the photo-z statistics as a function of photo-z bin for a _runid_ (from stats.dat).
By default, three plots for three statistics are made: robust standard deviation, robust bias, and fraction of outliers.
The default is to apply catastrophic outlier rejection (COR) to the robust standard deviation and robust bias statistics (COR is defined by user input parameter stats\_COR for cmnn\_run.run).

**Outputs:** Saves plots to the output/run\__runid_/analysis/ directory (CORIQRstdd.png, CORIQRbias.png, fout.png).

**Options:** When run stand-alone (i.e., not from cmnn\_run.run), the user may specify which of the statistical measures of photo-z quality a plot should be created for, and/or whether multiple _runids_ be co-plotted. When multiple _runids_ are co-plotted, the plots are saved to the directory "output/stats_plots/" with names formatted like "fout\_runid1\_runid2\_runid3.png". The Examples above demonstrate how to plot the statistical results from multiple runs into a single figure using the options in the table below.


##### Table 4: Additional Arguments for Making Multi-Run Plots of Statistical Measures
| Inputs | Type | Default | Description |
| :-- | :-- | :-- | :-- |
| runid              | str    | None | single runid to plot `user_stats` for |
| user\_stats        | str[M] | ['fout', 'CORIQRstdd', 'CORIQRbias'] | list of statistics for which to make plots |
| statsfile\_suffix  | str    | None | if not None, read file "stats_[suffix].dat" and add suffix to output plot names |
| bin\_in\_truez     | bool   | False | if True, plot vs. true z (read file "stats_truezbins.dat" not "stats.dat") |
| show\_SRD          | bool   | True | if True, the SRD target values are shown as dashed horizontal lines (ls.st/lpm-17) |
| show\_binw         | bool   | True | if True, thin horizontal bars are plotted to show the redshift bins |
| multi\_run\_ids    | str[N] | ['runid1', 'runid2'] | array of N run ids to co-plot |
| multi\_run\_labels | str[N] | ['label1', 'label2'] | array of N legend labels that describe each run |
| multi\_run\_colors | str[N] | ['black', 'orange', 'green', 'darkviolet', 'blue', 'red'] | colors to use for each run, must have at least N color names |
| plot_title         | str    | '' | add a title to the plot | 
| custom_pfnm_id     | str    | None | custom plot filename identifier (only for use instead of multiple runids) |
| custom_axlims      | float[2,2] | None | custom axes limits [[x1, x2], [y1, y2]] |

**Notes:**

Allowable statistics to request plots for `user_stats` includes: 
'fout', 'stdd', 'bias', 'outbias', 'IQR', 'IQRstdd', 'IQRbias', 'CORstdd', 'CORbias', 'CORoutbias', 'CORIQR', 'CORIQRstdd', 'CORIQRbias'.

If multiple run ids are passed, the resulting statisistical comparison plots will be named, e.g., `output/stats_plots/fout_runid1_runid1.png` or if `custom_pfnm_id` is also given, then `output/stats_plots/fout_custom_pfnm_id.png'.

#### 3.4.3. `cmnn_analysis.make_tzpz_plot`

Called by `cmnn_run.run`.

**What it does:** Plots the true (y-axis) vs. the photometric redshifts (x-axis) as a 2d histogram.
Includes user-specified options to represent outliers with colored points, to draw polygons, and/or to plot phot/true redshifts on the y/x axes instead.

**Outputs:** Saves plot to output/run\__runid_/analysis/tzpz.png (or pztz.png if the user decides to swap the axes).

#### 3.4.4. `cmnn_analysis.make_hist_plots`

Called by `cmnn_run.run`.

**What it does:** Plots histograms of the results: photometric redshifts (compared with true redshifts), size of the CMNN subset of training galaxies, and size of the training set used (after any color or magnitude cuts).

**Outputs:** Saves plot to the output/run\__runid_/analysis/ directory (hist\_ncm.png, hist\_ntr.png, hist\_z.png).

#### 3.4.5. `cmnn_analysis.get_stats`

Called by `cmnn_analysis.make_stats_file`.

**What it does:** When passed a full set of true and photometric redshifts and a desired redshift bin size, calculates all statistical measures (Table 3) for that redshift bin.

**Returns:** A set of values for all of the statistical measures in Table 3 (i.e., from column 2 `meanz` and up).

### 3.5. `cmnn_tools`

#### 3.5.1. `cmnn_tools.convert_visits_to_depths`

**What it does:** Converts the number of standard 30 second visits to be done in each filter _ugrizy_ into a 5$\sigma$ limiting magnitude depth for each filter.<br>

**Returns:** The 5$\sigma$ limiting magnitude depths in _ugrizy_ as a 6-element array.

**Example:**
```
>>> import cmnn_tools
>>> results_A = cmnn_tools.convert_visits_to_depths( [28, 40, 92, 92, 80, 80] )
>>> print(results_A)
[ 25.70894754  27.00257499  27.15473478  26.45473478  25.67886248 24.47886248]
```

#### 3.5.2. `cmnn_tools.convert_year_to_depths_baseline`

**What it does:** Convert the year of a survey into the $5{\sigma}$ limiting magnitudes depths in _ugrizy_, assuming a baseline LSST observing strategy.<br>

**Returns:** the $5{\sigma}$ limiting magnitudes depths in _ugrizy_ as a 6-element array.

**Example:**
```
>>> import cmnn_tools
>>> results_B = cmnn_tools.convert_year_to_depths_baseline( 5 )
>>> print(results_B)
[ 25.70894754  27.00257499  27.15473478  26.45473478  25.67886248  24.47886248]
```